# Exercise 11
## Time Series Analysis & Recurrent Neural Networks, SoSe 2021
### Author: Elias Olofsson
    Version information:
        2021-07-05: v.1.0. First public release. 

## Task 1: Laplace Approximation

Laplace's method states that
    \begin{equation}
        \int e^{M f(x)} \approx \sqrt{\frac{2 \pi}{M\left|f^{\prime \prime}\left(x_{0}\right)\right|}} e^{M f\left(x_{0}\right)} \text { as } M \rightarrow \infty,
    \end{equation}
    with $x_0$ as the global maximum of $f$. Using the Laplace's method, we can approximate the integral
    \begin{equation}
        N ! =\int_{0}^{\infty} e^{-t} t^{N} d t.
    \end{equation}
    First, we notice that the integral can be rewritten as
    \begin{align}
        N ! &=\int_{0}^{\infty} e^{-t} t^{N} d t\\
        &= \int_{0}^{\infty} e^{-t} e^{\log(t^{N})} d t\\
        &= \int_{0}^{\infty} e^{N\log(t)-t} d t.
    \end{align}
    Doing a variable substitution $t = Nx$, $dt = N dx$, we get
    \begin{align}
        N ! &= \int_{0}^{\infty} e^{N\log(N)+ N\log(x)-Nx} N d x\\
        &= N^{N+1}\int_{0}^{\infty} e^{N\log(x)-Nx} d x\\
        &= N^{N+1}\int_{0}^{\infty} e^{Nf(x)} d x, 
    \end{align}
    where we defined
    \begin{equation}
        f(x) = \log(x)-x.
    \end{equation}
    We can notice that $f$ is only defined for $x>0$ and has a single extrema at $x_0 = 1$, which is the global maximum. Doing a Taylor expansion of $f$ at this extreme point gives
    \begin{align}
        f(x) &= f(x_0) + f'(x_0)(x-x_0) - \frac{1}{2} f''(x_0)(x-x_0)^2 + \mathcal{O}((x-x_0)^3) \\
        &\approx f(x_0) - \frac{1}{2} |f''(x_0)|(x-x_0)^2,
    \end{align}
    where the first derivative disappeared since $x_0$ is an extrema. Plugging back into the integral yields
    \begin{align}
        N ! &\approx N^{N+1}\int_{0}^{\infty} e^{Nf(x_0) - \frac{1}{2} N|f''(x_0)|(x-x_0)^2} d x\\
        &= N^{N+1} e^{Nf(x_0)}\int_{0}^{\infty} e^{- \frac{1}{2} N|f''(x_0)|(x-x_0)^2} d x\\
        &\approx N^{N+1} e^{Nf(x_0)}\int_{-\infty}^{\infty} e^{-\frac{1}{2} N|f''(x_0)|(x-x_0)^2} d x
    \end{align}
    where the last line is only approximately true since technically $f$ is only defined for $x>0$ in the original functional expression. However, since $x_0 = 1 > 0$ and N is large (i.e. small width/variance), the integral will have a very small contribution from the region $x<0$. Nonetheless, this extension of the integration bounds allows us to manipulate the integral and complete it into a Gaussian integral with mean $\mu = x_0$ and variance $\sigma^2 = 1/\left(N|f''(t_0)|\right)$ as per
    \begin{align}
        N ! &\approx N^{N+1} \sqrt{\frac{2\pi}{N|f''(x_0)|}} e^{Nf(x_0)} \int_{-\infty}^{\infty} \sqrt{\frac{N|f''(x_0)|}{2\pi}} e^{-\frac{1}{2}N|f''(x_0)|(x-x_0)^2} d x\\
        &= N^{N+1} \sqrt{\frac{2\pi}{N|f''(x_0)|}} e^{Nf(x_0)}
    \end{align}
    where we used the fact that the Gaussian is normalized. Finally, plugging in the function value and the second derivative of $f(x)$ at $x_0 = 1$, we arrive at
    \begin{align}
        N ! &\approx N^{N+1} \sqrt{\frac{2\pi}{N|f''(x_0)|}} e^{Nf(x_0)}\\
        &= N^{N+1} \sqrt{\frac{2\pi}{N}} e^{-N}\\
        &= \sqrt{2\pi N} N^N e^{-N}\\
        &= \sqrt{2\pi N} \left(\frac{N}{e}\right)^N,
    \end{align}
    which is valid as $N \rightarrow \infty$, also known as Sterling's approximation.

## Task 2: Extended Kalman Filter

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#plt.rc('image', cmap='gray')

import torch as tc
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Notebook graphics settings: 
%config InlineBackend.figure_format = 'svg' # inline graphics (options: 'svg', 'png', 'retina', etc.)
plt.rcParams['figure.dpi'] = 200            # custom dpi setting for inline png:s